In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
suicide = pd.read_csv('/kaggle/input/suicide-rates-overview-1985-to-2016/master.csv')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
suicide.head()

In [ ]:
plt.plot(figsize = (20,12))
sns.distplot(suicide['suicides/100k pop'])

In [ ]:
plt.plot(figsize = (20,12))
sns.distplot(suicide['suicides_no'])

In [ ]:
suicide.columns

In [ ]:
plt.figure(figsize = (20,12))
sns.boxplot(suicide['country'],suicide['suicides/100k pop'])

In [ ]:
suicide['generation'].unique()

In [ ]:
s_gx = suicide[ suicide['generation'] == 'Generation X']
s_si = suicide[ suicide['generation'] == 'Silent']
s_gi = suicide[ suicide['generation'] == 'G.I. Generation']
s_bo = suicide[ suicide['generation'] == 'Boomers']
s_mi = suicide[ suicide['generation'] == 'Millenials']
s_gz = suicide[ suicide['generation'] == 'Generation Z']

In [ ]:
plt.figure(figsize = (20,12))
sns.boxplot(s_gx['country'],s_gx['suicides/100k pop'])

In [ ]:
suicide.plot(' gdp_for_year ($) ','suicides/100k pop',figsize = (20,12))

In [ ]:
suicide.plot('gdp_per_capita ($)','suicides/100k pop',figsize = (20,12))

In [ ]:
plt.figure(figsize = (20,12) )
sns.boxplot(data = suicide, x = 'age', y = 'suicides/100k pop', hue = 'sex')

In [ ]:
plt.figure(figsize = (16,8))
sns.countplot(data = suicide, x = 'sex')

In [ ]:
suicide.isna().sum()

In [ ]:
suicide[ suicide['country'] == 'Albania'].tail(50)

In [ ]:
for i in suicide['country'].unique():
    suicide = suicide.fillna( suicide[ suicide['country'] == i]['HDI for year'].median() )

In [ ]:
suicide.isna().sum()

In [ ]:
suicide.info()

In [ ]:
suicide[' gdp_for_year ($) '] = suicide[' gdp_for_year ($) '].str.replace(",","")
suicide[' gdp_for_year ($) '] = suicide[' gdp_for_year ($) '].astype('int64')
suicide

In [ ]:
suicide.info()

In [ ]:
suicide.corr()

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for i in suicide.columns[ suicide.dtypes == object]:
    suicide[i] = le.fit_transform( suicide[i] )
suicide.info()

In [ ]:
suicide.corr()[ sucide.corr() > 0.1]

In [ ]:
suicide['gdp_per_pop'] = suicide[' gdp_for_year ($) ']/suicide['population']


In [ ]:
suicide.corr()

In [ ]:
suicide = suicide.drop(['suicides_no',' gdp_for_year ($) '],1)
suicide.corr()[ suicide.corr() > 0.35] 

In [ ]:
suicide = suicide.drop('country',1)
suicide.corr()[ suicide.corr() > 0.35] 

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
suicide_st = sc.fit_transform(suicide)

In [ ]:
from sklearn.model_selection import train_test_split
y = suicide['suicides/100k pop']
x = suicide.drop('suicides/100k pop',1)
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.3)

In [ ]:
from catboost import CatBoostRegressor
cb = CatBoostRegressor(subsample = 0.8, colsample_bylevel=0.8, learning_rate=0.8, n_estimators=800)

In [ ]:
# cb.fit(x_train,np.log(y_train))
cb.fit(x_train,y_train)

In [ ]:
result = cb.predict(x_test)

In [ ]:
pd.Series(cb.feature_importances_, index = suicide.drop('suicides/100k pop',1).columns).sort_values(ascending = False)

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
mse = mean_squared_error(y_test,result)
rmse = np.sqrt(mse)

In [ ]:
rmse